# پروژه تشخیص پلاک خودروهای ایرانی با استفاده از YOLO

## نمای کلی پروژه

### اهداف پروژه

این پروژه با هدف طراحی و پیاده‌سازی یک سیستم تشخیص پلاک خودروهای ایرانی بر روی یک سیستم embedded (رزبری پای) انجام شده است. برخلاف اکثر سیستم‌های موجود که در تشخیص پلاک‌های خاص (مانند پلاک‌های دیپلمات، سرویس، معلولین و ...) با مشکل مواجه هستند، ما سعی کردیم این مشکل را با استفاده از یک رویکرد یکپارچه و افزایش داده‌های هدفمند حل کنیم.

### نوآوری‌های پروژه

1. **رویکرد یکپارچه**: به جای استفاده از دو مدل جداگانه (یکی برای تشخیص پلاک و دیگری برای تشخیص کاراکترها)، ما از یک مدل YOLO واحد برای همزمان تشخیص محدوده پلاک و تمام کاراکترهای آن استفاده کردیم.

2. **پوشش پلاک‌های خاص**: مدل ما قادر به تشخیص 41 کلاس مختلف شامل پلاک، اعداد (0-9)، حروف فارسی و نمادهای خاص است.

3. **بهینه‌سازی برای سخت‌افزار محدود**: پیاده‌سازی بر روی Raspberry Pi 5 با بهینه‌سازی‌های لازم برای اجرا با فریم‌ریت قابل قبول.

### کلاس‌های تعریف شده در مدل

مدل ما شامل 41 کلاس است:
- **plate**: ناحیه کلی پلاک
- **اعداد**: 0 تا 9
- **حروف فارسی**: الف، ب، پ، ت، ث، ج، د، ز، س، ش، ص، ط، ظ، ع، ف، ق، ک، گ، ل، م، ن، و، ه، ی، ژ
- **نمادهای خاص**: تاکسی، معلولین، پلیس، دیپلمات (D)، سرویس (S)، تشریفات

### مراحل انجام پروژه

1. جمع‌آوری و پردازش دیتاست IR-LPR
2. تبدیل فرمت annotations از XML به YOLO
3. افزایش داده برای کلاس‌های کم‌نمونه (Data Augmentation)
4. آموزش مدل YOLOv8n
5. ارزیابی و تست مدل
6. بهینه‌سازی برای Raspberry Pi (تبدیل به NCNN)
7. پیاده‌سازی نهایی با دوربین RPi Camera

---

## بخش اول: آماده‌سازی دیتاست

### توضیحات

دیتاست IR-LPR که از GitHub دریافت شده، شامل بیش از 76,000 تصویر با annotations در فرمت XML است. این دیتاست دارای مشکلات زیر بود:

1. **فرمت نامناسب**: YOLO نیاز به فرمت خاص خود (class x_center y_center width height) دارد در حالی که دیتاست در فرمت XML با مختصات (xmin, ymin, xmax, ymax) بود.

2. **عدم تعادل کلاس‌ها**: از 76,000 تصویر، کمتر از 3,000 تصویر شامل پلاک‌های خاص (دیپلمات، معلولین، تشریفات و ...) بودند.

### راه‌حل

دو اسکریپت اصلی برای حل این مشکلات نوشته شد:
- **Dataset**: تبدیل فرمت و سازماندهی دیتاست
- **Augmentation**: افزایش داده برای کلاس‌های کم‌نمونه

### عملکرد dataset

این اسکریپت وظایف زیر را انجام می‌دهد:
1. خواندن فایل‌های XML از سه پوشه منبع (car_img, plate_img, plate_img_dummy)
2. تبدیل مختصات bounding box از فرمت (xmin, ymin, xmax, ymax) به فرمت YOLO نرمالیزه شده
3. نگاشت برچسب‌های فارسی به شماره کلاس‌های انگلیسی
4. سازماندهی داده‌ها در ساختار استاندارد YOLO:
   ```
   yolo_dataset/
   ├── images/
   │   ├── train/
   │   ├── val/
   │   └── test/
   ├── labels/
   │   ├── train/
   │   ├── val/
   │   └── test/
   └── data.yaml
   ```
5. ایجاد فایل data.yaml برای آموزش YOLO

# Dataset
dataset.py

In [ ]:
import os
import shutil
import xml.etree.ElementTree as ET
import cv2
import yaml
from pathlib import Path

# ==========================================
# CONFIGURATION
# ==========================================

SOURCE_DIRS = [
    "car_img",
    "plate_img",
    "plate_img_dummy"
]

OUTPUT_DIR = "yolo_dataset"

# Mapping original folder names to YOLO standard splits
SPLIT_MAPPING = {
    "train": "train",
    "test": "test",
    "validation": "val"
}

# Define all 41 classes in English
CLASS_NAMES = [
    "plate",      # Full plate region
    "0", "1", "2", "3", "4", "5", "6", "7", "8", "9",  # Digits
    "alef",       # ا
    "be",         # ب
    "pe",         # پ
    "te",         # ت
    "se",         # ث
    "jim",        # ج
    "dal",        # د
    "sin",        # س
    "shin",       # ش
    "sad",        # ص
    "ta",         # ط
    "za",         # ظ
    "ein",        # ع
    "fe",         # ف
    "ghaf",       # ق
    "kaf",        # ک
    "gaf",        # گ
    "lam",        # ل
    "mim",        # م
    "noon",       # ن
    "vav",        # و
    "he",         # ه
    "ye",         # ی
    "ze",         # ز
    "taxi",       # تاکسی
    "disabled",   # معلولین
    "police",     # پلیس
    "D",          # Diplomatic
    "S",          # Service
    "protocol",   # تشریفات
]

# Mapping Persian labels to class indices
LABEL_MAP = {
    "کل ناحیه پلاک": CLASS_NAMES.index("plate"),
    # Numbers
    "0": CLASS_NAMES.index("0"), "1": CLASS_NAMES.index("1"), 
    "2": CLASS_NAMES.index("2"), "3": CLASS_NAMES.index("3"),
    "4": CLASS_NAMES.index("4"), "5": CLASS_NAMES.index("5"), 
    "6": CLASS_NAMES.index("6"), "7": CLASS_NAMES.index("7"),
    "8": CLASS_NAMES.index("8"), "9": CLASS_NAMES.index("9"),
    # Persian alphabets
    "الف": CLASS_NAMES.index("alef"),
    "ب": CLASS_NAMES.index("be"),
    "پ": CLASS_NAMES.index("pe"),
    "ت": CLASS_NAMES.index("te"),
    "ث": CLASS_NAMES.index("se"),
    "ج": CLASS_NAMES.index("jim"),
    "د": CLASS_NAMES.index("dal"),
    "س": CLASS_NAMES.index("sin"),
    "ش": CLASS_NAMES.index("shin"),
    "ص": CLASS_NAMES.index("sad"),
    "ط": CLASS_NAMES.index("ta"),
    "ظ": CLASS_NAMES.index("za"),
    "ع": CLASS_NAMES.index("ein"),
    "ف": CLASS_NAMES.index("fe"),
    "ق": CLASS_NAMES.index("ghaf"),
    "ک": CLASS_NAMES.index("kaf"),
    "گ": CLASS_NAMES.index("gaf"),
    "ل": CLASS_NAMES.index("lam"),
    "م": CLASS_NAMES.index("mim"),
    "ن": CLASS_NAMES.index("noon"),
    "و": CLASS_NAMES.index("vav"),
    "ه‌": CLASS_NAMES.index("he"),
    "ه": CLASS_NAMES.index("he"),
    "ی": CLASS_NAMES.index("ye"),
    "ز": CLASS_NAMES.index("ze"),
    # Special symbols
    "ژ (معلولین و جانبازان)": CLASS_NAMES.index("disabled"),
    "تشریفات": CLASS_NAMES.index("protocol"),
    "S": CLASS_NAMES.index("S"),
    "D": CLASS_NAMES.index("D"),
}

# ==========================================
# HELPER FUNCTIONS
# ==========================================

def convert_box(size, box):
    """
    Convert bounding box from (xmin, ymin, xmax, ymax) to YOLO format.
    YOLO format: (x_center, y_center, width, height) - all normalized to [0,1]
    
    Args:
        size: tuple (image_width, image_height)
        box: tuple (xmin, xmax, ymin, ymax)
    
    Returns:
        tuple: (x_center, y_center, width, height) normalized
    """
    dw = 1.0 / size[0]
    dh = 1.0 / size[1]
    
    x_center = (box[0] + box[1]) / 2.0
    y_center = (box[2] + box[3]) / 2.0
    w = box[1] - box[0]
    h = box[3] - box[2]
    
    # Normalize to [0,1]
    x_center = x_center * dw
    w = w * dw
    y_center = y_center * dh
    h = h * dh
    
    return (x_center, y_center, w, h)


def process_dataset():
    """
    Main function to process the entire dataset:
    1. Create YOLO directory structure
    2. Convert XML annotations to YOLO format
    3. Copy images and create label files
    4. Generate data.yaml config file
    """
    print("Starting dataset conversion...")
    
    # Create output directory structure
    for split in ["train", "val", "test"]:
        os.makedirs(os.path.join(OUTPUT_DIR, "images", split), exist_ok=True)
        os.makedirs(os.path.join(OUTPUT_DIR, "labels", split), exist_ok=True)

    # Process each source directory
    for source_dir in SOURCE_DIRS:
        if not os.path.exists(source_dir):
            print(f"Warning: Directory '{source_dir}' not found. Skipping.")
            continue

        # Process each data split (train/val/test)
        for original_split, yolo_split in SPLIT_MAPPING.items():
            current_path = os.path.join(source_dir, original_split)
            
            if not os.path.exists(current_path):
                continue
            
            print(f"Processing: {current_path} -> {yolo_split}")
            
            files = os.listdir(current_path)
            # Get unique filenames (without extension) for pairing XML and JPG
            filenames = set([os.path.splitext(f)[0] for f in files if f.endswith('.xml')])

            for name in filenames:
                jpg_file = os.path.join(current_path, name + ".jpg")
                xml_file = os.path.join(current_path, name + ".xml")
                
                # Verify image exists
                if not os.path.exists(jpg_file):
                    print(f"Missing image for {xml_file}")
                    continue
                
                # Read image to get actual dimensions
                img = cv2.imread(jpg_file)
                if img is None:
                    print(f"Corrupt image: {jpg_file}")
                    continue
                
                h, w, _ = img.shape
                
                # Parse XML annotation
                try:
                    tree = ET.parse(xml_file)
                    root = tree.getroot()
                except ET.ParseError:
                    print(f"XML Parse Error: {xml_file}")
                    continue

                yolo_lines = []
                
                # Extract each object annotation
                for obj in root.findall('object'):
                    class_name_raw = obj.find('name').text.strip()
                    
                    # Map Persian label to class ID
                    if class_name_raw in LABEL_MAP:
                        class_id = LABEL_MAP[class_name_raw]
                    else:
                        print(f"Warning: Unknown label '{class_name_raw}' in {xml_file}. Skipping.")
                        continue
                    
                    # Extract bounding box coordinates
                    xml_box = obj.find('bndbox')
                    b = (
                        float(xml_box.find('xmin').text),
                        float(xml_box.find('xmax').text),
                        float(xml_box.find('ymin').text),
                        float(xml_box.find('ymax').text)
                    )
                    
                    # Convert to YOLO format
                    bb = convert_box((w, h), b)
                    yolo_lines.append(f"{class_id} {bb[0]:.6f} {bb[1]:.6f} {bb[2]:.6f} {bb[3]:.6f}")

                # Save only if valid objects were found
                if yolo_lines:
                    # Create unique filename to avoid conflicts
                    unique_name = f"{source_dir}_{original_split}_{name}"
                    
                    target_img_path = os.path.join(OUTPUT_DIR, "images", yolo_split, unique_name + ".jpg")
                    target_lbl_path = os.path.join(OUTPUT_DIR, "labels", yolo_split, unique_name + ".txt")
                    
                    # Copy image
                    shutil.copy(jpg_file, target_img_path)
                    
                    # Write label file
                    with open(target_lbl_path, 'w', encoding='utf-8') as f:
                        f.write('\n'.join(yolo_lines))

    print("Dataset conversion completed successfully.")
    create_yaml()


def create_yaml():
    """
    Generate the data.yaml configuration file required by YOLO training.
    Contains paths, number of classes, and class names.
    """
    yaml_content = {
        'path': os.path.abspath(OUTPUT_DIR),
        'train': 'images/train',
        'val': 'images/val',
        'test': 'images/test',
        'nc': len(CLASS_NAMES),
        'names': CLASS_NAMES
    }
    
    yaml_path = os.path.join(OUTPUT_DIR, "data.yaml")
    with open(yaml_path, 'w') as f:
        yaml.dump(yaml_content, f, sort_keys=False)
    
    print(f"Created config file at: {yaml_path}")


if __name__ == "__main__":
    process_dataset()

---

## بخش دوم: افزایش داده (Data Augmentation)

### مشکل عدم تعادل کلاس‌ها

یکی از چالش‌های اصلی در آموزش مدل، عدم تعادل شدید بین کلاس‌های مختلف بود. به طور مثال:
- کلاس‌های حروف رایج (الف، ب، د): 70,000 نمونه
- کلاس‌های خاص (پ، ث، ش، ظ، گ، ک، معلولین، تشریفات، D، S): کمتر از 3,000 نمونه

این عدم تعادل باعث می‌شود مدل در تشخیص کلاس‌های کم‌نمونه عملکرد ضعیفی داشته باشد.

### راه‌حل: افزایش داده هدفمند

برای حل این مشکل، یک سیستم افزایش داده هدفمند طراحی شد که:

1. **هدف‌گیری هوشمند**: فقط کلاس‌هایی که زیر 5,000 نمونه دارند را افزایش می‌دهد
2. **حفظ یکپارچگی**: تصاویر را به همراه تمام bounding box های آن‌ها augment می‌کند
3. **تنوع بالا**: از ترکیب چندین تکنیک augmentation استفاده می‌کند
4. **جلوگیری از flip**: چون پلاک‌های ایران قابل قرینه شدن نیستند، از horizontal/vertical flip استفاده نمی‌شود

### تکنیک‌های Augmentation استفاده شده

- **چرخش (Rotation)**: تا 25 درجه (با احتمال 80%)
- **تغییر روشنایی و کنتراست**: ±20% (احتمال 50%)
- **تغییر رنگ (Hue/Saturation)**: تغییرات جزئی (احتمال 40%)
- **افزودن نویز Gaussian**: (احتمال 30%)
- **Motion Blur**: برای شبیه‌سازی حرکت (احتمال 20%)

### نتیجه

پس از اجرای این فرآیند، دیتاست از 76,000 به حدود 105,000 تصویر افزایش یافت و تمام کلاس‌های هدف به حداقل 5,000 نمونه رسیدند.

## Augmentation
aug.py

In [ ]:
import os
import cv2
import glob
import random
import numpy as np
import albumentations as A
from tqdm import tqdm

# ==========================================
# CONFIGURATION
# ==========================================

DATASET_DIR = "yolo_dataset"
TARGET_COUNT = 5000  # Minimum required instances per target class

# All 41 classes defined in the model
CLASS_NAMES = [
    "plate",      
    "0", "1", "2", "3", "4", "5", "6", "7", "8", "9", 
    "alef", "be", "pe", "te", "se", "jim", "dal", "ze", "sin", 
    "shin", "sad", "ta", "za", "ein", "fe", "ghaf", "kaf", 
    "gaf", "lam", "mim", "noon", "vav", "he", "ye", "zhe", 
    "disabled", "protocol", "S", "D",
]

# Classes that need augmentation (underrepresented in dataset)
TARGET_CLASSES_NAMES = [
    "alef", "pe", "se", "shin", "za", "kaf", "gaf", 
    "disabled", "protocol", "S", "D"
]

# Convert class names to indices
TARGET_CLASS_IDS = [CLASS_NAMES.index(name) for name in TARGET_CLASSES_NAMES if name in CLASS_NAMES]

# ==========================================
# AUGMENTATION PIPELINE
# ==========================================

"""
Augmentation pipeline designed specifically for license plates:
- NO horizontal/vertical flips (Iranian plates are not symmetric)
- Moderate rotation (up to 25 degrees)
- Color and brightness variations to simulate different lighting
- Noise and blur to simulate real-world conditions
"""
aug_pipeline = A.Compose([
    A.Rotate(limit=25, border_mode=cv2.BORDER_CONSTANT, value=0, p=0.8),
    A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5),
    A.HueSaturationValue(hue_shift_limit=10, sat_shift_limit=20, val_shift_limit=10, p=0.4),
    A.GaussNoise(var_limit=(10.0, 50.0), p=0.3),
    A.MotionBlur(blur_limit=3, p=0.2),
], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels']))

# ==========================================
# HELPER FUNCTIONS
# ==========================================

def read_yolo_label(txt_path):
    """
    Read YOLO format label file and extract bounding boxes and class IDs.
    
    Args:
        txt_path: Path to .txt label file
    
    Returns:
        boxes: List of bounding boxes in YOLO format [x_center, y_center, width, height]
        class_labels: List of corresponding class IDs
    """
    boxes = []
    class_labels = []
    
    if not os.path.exists(txt_path):
        return boxes, class_labels
        
    with open(txt_path, 'r') as f:
        lines = f.readlines()
        
    for line in lines:
        parts = line.strip().split()
        if len(parts) == 5:
            cls_id = int(parts[0])
            bbox = [float(x) for x in parts[1:]]  # x_center, y_center, width, height
            
            boxes.append(bbox)
            class_labels.append(cls_id)
            
    return boxes, class_labels


def save_yolo_label(txt_path, boxes, class_labels):
    """
    Save bounding boxes and class labels to YOLO format text file.
    Ensures all values are properly clipped to [0,1] range.
    
    Args:
        txt_path: Output path for label file
        boxes: List of bounding boxes
        class_labels: List of class IDs
    """
    with open(txt_path, 'w') as f:
        for bbox, cls_id in zip(boxes, class_labels):
            # Clip values to valid range [0,1]
            xc, yc, w, h = bbox
            xc = max(0.0, min(1.0, xc))
            yc = max(0.0, min(1.0, yc))
            w = max(0.0, min(1.0, w))
            h = max(0.0, min(1.0, h))
            
            f.write(f"{cls_id} {xc:.6f} {yc:.6f} {w:.6f} {h:.6f}\n")

# ==========================================
# MAIN BALANCING PROCESS
# ==========================================

def balance_dataset():
    """
    Main function to balance the dataset by augmenting underrepresented classes.
    
    Process:
    1. Index all images containing target classes
    2. For each class below TARGET_COUNT:
       - Randomly select images containing that class
       - Apply augmentation pipeline
       - Save augmented images and labels
    3. Continue until all classes reach TARGET_COUNT
    """
    print(f"Target classes to augment: {TARGET_CLASSES_NAMES}")
    print(f"Target count per class: {TARGET_COUNT}")

    # Index dataset - map each class to images containing it
    splits = ['train', 'val', 'test']
    all_files_map = {cid: [] for cid in TARGET_CLASS_IDS}
    
    print("Indexing dataset...")
    for split in splits:
        img_dir = os.path.join(DATASET_DIR, "images", split)
        lbl_dir = os.path.join(DATASET_DIR, "labels", split)
        
        txt_files = glob.glob(os.path.join(lbl_dir, "*.txt"))
        
        for txt_file in tqdm(txt_files, desc=f"Scanning {split}"):
            _, class_ids_in_file = read_yolo_label(txt_file)
            
            # Find corresponding image
            basename = os.path.basename(txt_file).replace('.txt', '.jpg')
            img_path = os.path.join(img_dir, basename)
            
            if not os.path.exists(img_path):
                continue
                
            # Map classes to this image
            unique_classes = set(class_ids_in_file)
            for cid in unique_classes:
                if cid in TARGET_CLASS_IDS:
                    all_files_map[cid].append({
                        'img_path': img_path,
                        'txt_path': txt_file,
                        'split': split
                    })

    # Augment each class as needed
    print("\n--- Augmentation Status ---")
    
    for cid in TARGET_CLASS_IDS:
        current_count = len(all_files_map[cid])
        class_name = CLASS_NAMES[cid]
        
        if current_count >= TARGET_COUNT:
            print(f"Class '{class_name}' (ID {cid}): OK ({current_count} instances)")
            continue
            
        needed = TARGET_COUNT - current_count
        print(f"Class '{class_name}' (ID {cid}): Needs {needed} more (Current: {current_count})")
        
        if current_count == 0:
            print(f"  WARNING: No instances found for '{class_name}'. Cannot augment.")
            continue

        # Generate augmented samples
        generated_count = 0
        pbar = tqdm(total=needed, desc=f"Augmenting {class_name}")
        
        while generated_count < needed:
            # Randomly select source image containing this class
            source_data = random.choice(all_files_map[cid])
            
            src_img_path = source_data['img_path']
            src_txt_path = source_data['txt_path']
            
            # Load image
            image = cv2.imread(src_img_path)
            if image is None:
                continue
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            
            # Load labels
            bboxes, labels = read_yolo_label(src_txt_path)
            
            # Apply augmentation
            try:
                transformed = aug_pipeline(image=image, bboxes=bboxes, class_labels=labels)
                aug_img = transformed['image']
                aug_bboxes = transformed['bboxes']
                aug_labels = transformed['class_labels']
                
                # Skip if augmentation removed all boxes
                if len(aug_bboxes) == 0 and len(bboxes) > 0:
                    continue
                    
            except Exception as e:
                # Skip failed augmentations
                continue
                
            # Save augmented data
            img_dir = os.path.dirname(src_img_path)
            lbl_dir = os.path.dirname(src_txt_path)
            
            # Create unique filename
            base_name = os.path.splitext(os.path.basename(src_img_path))[0]
            new_name = f"{base_name}_aug_{generated_count}_{random.randint(100,999)}"
            
            target_img_path = os.path.join(img_dir, new_name + ".jpg")
            target_txt_path = os.path.join(lbl_dir, new_name + ".txt")
            
            # Write image (convert back to BGR for OpenCV)
            cv2.imwrite(target_img_path, cv2.cvtColor(aug_img, cv2.COLOR_RGB2BGR))
            
            # Write label
            save_yolo_label(target_txt_path, aug_bboxes, aug_labels)
            
            generated_count += 1
            pbar.update(1)
            
        pbar.close()

    print("\nData balancing complete.")


if __name__ == "__main__":
    balance_dataset()

---

## بخش سوم: آموزش مدل

### انتخاب مدل پایه

برای این پروژه از **YOLOv8n** (نسخه Nano) استفاده شد. دلایل این انتخاب:

1. **سبک‌وزن بودن**: با حدود 3.2 میلیون پارامتر، برای سخت‌افزار محدود مناسب است
2. **سرعت بالا**: قابلیت پردازش real-time حتی بر روی CPU
3. **دقت قابل قبول**: با وجود سبک بودن، دقت مناسبی برای detection دارد
4. **پشتیبانی عالی**: کتابخانه Ultralytics به‌روز و کامل است

### تنظیمات آموزش

**پارامترهای کلیدی:**
- **Epochs**: 20 (با early stopping patience=10)
- **Batch size**: 64
- **Image size**: 640×640
- **Device**: GPU (CUDA)

**غیرفعال‌سازی augmentation‌های نامناسب:**

YOLO به‌صورت پیش‌فرض از flip و mixup استفاده می‌کند که برای پلاک ایرانی مناسب نیست:
- `fliplr: 0.0` - چون پلاک قابل قرینه شدن نیست
- `flipud: 0.0` - مشابه بالا
- `mixup: 0.0` - ترکیب دو تصویر منطقی نیست
- `copy_paste: 0.0` - برای پلاک کاربردی نیست

### معیارهای ارزیابی

مدل بر اساس معیارهای زیر ارزیابی می‌شود:
- **Precision**: نسبت تشخیص‌های درست به کل تشخیص‌ها
- **Recall**: نسبت تشخیص‌های درست به کل موارد واقعی
- **mAP@50**: میانگین دقت در IoU threshold 0.5
- **mAP@50-95**: میانگین دقت در range مختلف IoU (معیار جامع‌تر)

## Train
train.py

In [ ]:
from ultralytics import YOLO
import torch
import os

# Check CUDA availability
print(f"CUDA Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU Device: {torch.cuda.get_device_name(0)}")
    print(f"CUDA Version: {torch.version.cuda}")

def train_model():
    """
    Train YOLOv8n model on Iranian license plate dataset.
    
    Configuration:
    - Base model: YOLOv8n (nano - lightweight for embedded systems)
    - Custom augmentations disabled (flips not suitable for plates)
    - Early stopping enabled (patience=10)
    - GPU training (device=0)
    """
    # Load pretrained YOLOv8n model
    model = YOLO('yolov8n.pt')
    
    # Path to dataset configuration
    yaml_path = os.path.join("yolo_dataset", "data.yaml")

    # Disable unsuitable augmentations for license plates
    custom_augs = {
        'fliplr': 0.0,      # No horizontal flip (plates are not symmetric)
        'flipud': 0.0,      # No vertical flip
        'mixup': 0.0,       # No mixup (doesn't make sense for plates)
        'copy_paste': 0.0,  # No copy-paste augmentation
    }

    print("Starting training...")
    results = model.train(
        data=yaml_path,
        epochs=20,
        imgsz=640,
        batch=64,
        name='iran_lpr_model',
        device=0,  # Use first GPU
        patience=10,  # Early stopping patience
        verbose=True,
        **custom_augs
    )

    print("Training finished.")
    
    # Validate the trained model
    print("\nValidating model...")
    metrics = model.val()
    print(f"\nValidation Results:")
    print(f"mAP@50-95: {metrics.box.map:.4f}")

if __name__ == '__main__':
    train_model()

---

## بخش چهارم: تست مدل

### فرآیند تست

برای ارزیابی عملکرد مدل در شرایط واقعی، یک اسکریپت تست طراحی شد که:

1. تصاویر تست را از یک پوشه می‌خواند
2. مدل را روی هر تصویر اجرا می‌کند
3. پلاک‌ها را شناسایی کرده و کاراکترهای داخل آن‌ها را استخراج می‌کند
4. کاراکترها را بر اساس موقعیت افقی مرتب می‌کند
5. متن پلاک را به فرمت استاندارد ایرانی نمایش می‌دهد
6. تصویر خروجی با bounding box ها و متن پلاک را ذخیره می‌کند

### فرمت نمایش پلاک

پلاک ایرانی به فرمت زیر نمایش داده می‌شود:
```
[دو رقم] [حرف] [سه رقم سریال] | [دو رقم شهر]
مثال: 12 | 345 ب 67
```

### توابع کمکی

- **is_inside()**: بررسی می‌کند آیا مرکز یک bounding box داخل bounding box دیگری است
- **format_plate_text()**: کاراکترها را به فرمت استاندارد پلاک تبدیل می‌کند

## Test
test.py

In [ ]:
import cv2
import os
from ultralytics import YOLO

# ==========================================
# CONFIGURATION
# ==========================================

MODEL_PATH = "model/best.pt"  # Path to trained model weights
INPUT_PATH = "test_model/"  # Folder containing test images
OUTPUT_FOLDER = "output_results"  # Output folder for annotated images

CLASS_NAMES = [
    "plate", "0", "1", "2", "3", "4", "5", "6", "7", "8", "9", 
    "alef", "be", "pe", "te", "se", "jim", "dal", "ze", "sin", 
    "shin", "sad", "ta", "za", "ein", "fe", "ghaf", "kaf", 
    "gaf", "lam", "mim", "noon", "vav", "he", "ye", "zhe", 
    "disabled", "protocol", "S", "D"
]

# Create output folder if it doesn't exist
if not os.path.exists(OUTPUT_FOLDER):
    os.makedirs(OUTPUT_FOLDER)

# ==========================================
# HELPER FUNCTIONS
# ==========================================

def is_inside(box_inner, box_outer):
    """
    Check if the center of box_inner is inside box_outer.
    Used to determine which characters belong to which plate.
    
    Args:
        box_inner: [x1, y1, x2, y2] of character box
        box_outer: [x1, y1, x2, y2] of plate box
    
    Returns:
        bool: True if center of inner box is inside outer box
    """
    c_x = (box_inner[0] + box_inner[2]) / 2
    c_y = (box_inner[1] + box_inner[3]) / 2
    return (box_outer[0] < c_x < box_outer[2]) and (box_outer[1] < c_y < box_outer[3])


def format_plate_text(chars_list):
    """
    Format detected characters into Iranian plate standard format.
    Format: [2 digits] [letter] [3 digits serial] | [2 digits city code]
    Example: 67 ب 345 | 12
    
    Args:
        chars_list: List of character classes sorted left to right
    
    Returns:
        str: Formatted plate text
    """
    if len(chars_list) < 8:
        # If less than 8 characters, return simple concatenation
        return " ".join(chars_list)
    
    # Standard Iranian plate format:
    # [0,1]: First two digits (left side)
    # [2]: Letter
    # [3,4,5]: Three middle digits
    # [6,7]: City code (right side)
    try:
        part1 = "".join(chars_list[0:2])    # First 2 digits
        part2 = "".join(chars_list[3:6])    # Middle 3 digits
        letter = chars_list[2]              # Letter
        part3 = "".join(chars_list[6:8])    # Last 2 digits (city code)
        
        return f"{part1} {letter} {part2} | {part3}"
    except:
        return " ".join(chars_list)

# ==========================================
# IMAGE PROCESSING
# ==========================================

def process_images():
    """
    Process all test images:
    1. Load images from INPUT_PATH
    2. Run YOLO inference
    3. Extract plates and characters
    4. Format and display plate text
    5. Save annotated images to OUTPUT_FOLDER
    """
    # Load trained model
    print(f"Loading model: {MODEL_PATH}")
    model = YOLO(MODEL_PATH)
    
    # Get list of images to process
    if os.path.isfile(INPUT_PATH):
        image_files = [INPUT_PATH]
    else:
        image_files = [os.path.join(INPUT_PATH, f) for f in os.listdir(INPUT_PATH) 
                       if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

    if not image_files:
        print("No images found in the specified path!")
        return

    print(f"Processing {len(image_files)} images...\n")

    for img_path in image_files:
        # Read image
        frame = cv2.imread(img_path)
        if frame is None:
            print(f"Failed to read: {img_path}")
            continue

        # Run YOLO inference (CPU mode by default)
        results = model(frame, verbose=False)[0]
        
        # Extract detections
        detections = []
        for box in results.boxes.data.tolist():
            x1, y1, x2, y2, score, class_id = box
            detections.append({
                'box': [int(x1), int(y1), int(x2), int(y2)],
                'class': CLASS_NAMES[int(class_id)],
                'score': score
            })

        # Separate plates and characters
        plates = [d for d in detections if d['class'] == 'plate']
        characters = [d for d in detections if d['class'] != 'plate']

        # Process each detected plate
        for plate in plates:
            px1, py1, px2, py2 = plate['box']
            
            # Find characters inside this plate
            plate_chars = [c for c in characters if is_inside(c['box'], plate['box'])]
            
            # Sort characters left to right
            plate_chars.sort(key=lambda x: x['box'][0])
            
            # Extract text
            text_list = [c['class'] for c in plate_chars]
            plate_text = format_plate_text(text_list)
            
            # Draw plate bounding box
            cv2.rectangle(frame, (px1, py1), (px2, py2), (0, 255, 0), 3)
            
            # Draw text label above plate
            label = f"{plate_text}"
            (w, h), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 2)
            
            # Draw background for text
            cv2.rectangle(frame, (px1, py1 - h - 15), (px1 + w, py1), (0, 165, 255), -1)
            
            # Draw text
            cv2.putText(frame, label, (px1, py1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

        # Save annotated image
        save_path = os.path.join(OUTPUT_FOLDER, os.path.basename(img_path))
        cv2.imwrite(save_path, frame)
        print(f"Processed: {os.path.basename(img_path)} -> Saved to {OUTPUT_FOLDER}")

    print("\nAll images processed successfully!")

if __name__ == "__main__":
    process_images()

---

## بخش پنجم: پیاده‌سازی روی Raspberry Pi

### چالش‌های پیاده‌سازی

پیاده‌سازی مدل روی Raspberry Pi 5 با چالش‌های زیر همراه بود:

#### 1. محدودیت‌های سخت‌افزاری
- **RAM**: 2GB (محدود برای مدل‌های deep learning)
- **CPU**: Quad-core ARM Cortex-A76 (بدون GPU قوی)
- **حافظه**: نبود SD Card با ظرفیت کافی

**راه‌حل**: بوت از USB Flash با ظرفیت بالاتر پس از آپدیت EEPROM

#### 2. مشکلات دوربین
دوربین RPi Camera Module از طریق پورت MIPI CSI متصل می‌شود و با کتابخانه‌های استاندارد OpenCV کار نمی‌کند.

**راه‌حل**: استفاده از کتابخانه PiCamZero که interface ساده‌ای برای دسترسی به libcamera فراهم می‌کند

#### 3. بهینه‌سازی مدل
مدل PyTorch (.pt) برای CPU معمولی بهینه نیست.

**راه‌حل**: تبدیل وزن‌های مدل به فرمت NCNN که برای ARM CPUs بهینه‌سازی شده است:
```bash
yolo export model=best.pt format=ncnn
```

### معماری سیستم نهایی

```
RPi Camera Module
       ↓
MIPI CSI Interface
       ↓
PiCamZero Library
       ↓
NumPy Array (640×480)
       ↓
YOLOv8n-NCNN Model
       ↓
Detection Results
       ↓
Console Output (FPS: ~1.5)
```

### عملکرد نهایی

- **FPS**: حدود 1.5 فریم در ثانیه
- **دقت**: مشابه مدل اصلی
- **تأخیر**: حدود 650-700 میلی‌ثانیه برای هر فریم

این سرعت برای یک سیستم پارکینگ که ماشین‌ها با سرعت کم حرکت می‌کنند، کاملاً قابل قبول است.

## Raspberry Pi
rpi.py

In [ ]:
import cv2
import time
import sys
from picamzero import Camera  # Raspberry Pi camera library
from ultralytics import YOLO

# ==========================================
# CONFIGURATION
# ==========================================

MODEL_PATH = "best_ncnn_model_640"  # NCNN optimized model for ARM CPU

CLASS_NAMES = [
    "plate", "0", "1", "2", "3", "4", "5", "6", "7", "8", "9", 
    "alef", "be", "pe", "te", "se", "jim", "dal", "ze", "sin", 
    "shin", "sad", "ta", "za", "ein", "fe", "ghaf", "kaf", 
    "gaf", "lam", "mim", "noon", "vav", "he", "ye", "zhe", 
    "disabled", "protocol", "S", "D",
]

# ==========================================
# HELPER FUNCTIONS
# ==========================================

def is_inside(box_inner, box_outer):
    """
    Check if center of inner box is inside outer box.
    Used to associate characters with their parent plate.
    """
    c_x = (box_inner[0] + box_inner[2]) / 2
    c_y = (box_inner[1] + box_inner[3]) / 2
    return (box_outer[0] < c_x < box_outer[2]) and (box_outer[1] < c_y < box_outer[3])


def format_plate_text(chars_list):
    """
    Simple concatenation of detected characters.
    Returns a readable string of the plate.
    """
    return "".join(chars_list)

# ==========================================
# MAIN PROCESS
# ==========================================

def main():
    """
    Main loop for real-time license plate recognition on Raspberry Pi.
    
    Process:
    1. Initialize RPi Camera using PiCamZero
    2. Load NCNN-optimized YOLO model
    3. Continuous frame capture and inference
    4. Extract and display detected plates
    5. Calculate and display FPS
    """
    print("--- License Plate Recognition with PiCamZero ---")
    
    # Initialize Raspberry Pi Camera
    try:
        cam = Camera()
        # PiCamZero handles libcamera configuration automatically
        print("Camera initialized successfully.")
    except Exception as e:
        print(f"FATAL: Could not initialize camera. Error: {e}")
        return

    # Load NCNN-optimized model
    print(f"Loading model: {MODEL_PATH}")
    try:
        model = YOLO(MODEL_PATH, task='detect')
        print("Model loaded successfully.")
    except Exception as e:
        print(f"FATAL: Model load error: {e}")
        return

    print("Starting Detection Loop. Press Ctrl+C to stop.\n")

    prev_time = time.time()

    try:
        while True:
            # Capture frame from camera
            # capture_array() returns numpy array compatible with OpenCV
            frame = cam.capture_array()

            # Calculate FPS
            curr_time = time.time()
            fps = 1 / (curr_time - prev_time)
            prev_time = curr_time

            # Run YOLO inference
            # conf=0.5: minimum confidence threshold
            # verbose=False: suppress detailed output
            results = model(frame, verbose=False, conf=0.5)[0]

            # Extract detections
            detections = []
            for box in results.boxes.data.tolist():
                x1, y1, x2, y2, score, class_id = box
                detections.append({
                    'box': [int(x1), int(y1), int(x2), int(y2)],
                    'class': CLASS_NAMES[int(class_id)]
                })

            # Separate plates and characters
            plates = [d for d in detections if d['class'] == 'plate']
            characters = [d for d in detections if d['class'] != 'plate']

            # Process each detected plate
            if plates:
                for plate in plates:
                    # Find characters belonging to this plate
                    plate_chars = [c for c in characters if is_inside(c['box'], plate['box'])]
                    
                    # Sort characters left to right
                    plate_chars.sort(key=lambda x: x['box'][0])
                    
                    # Format plate text
                    text = format_plate_text([c['class'] for c in plate_chars])
                    
                    if text:
                        # Print detection with timestamp and FPS
                        print(f"[{time.strftime('%H:%M:%S')}] PLATE DETECTED: {text} (FPS: {fps:.1f})")

    except KeyboardInterrupt:
        print("\nStopping script...")
    finally:
        # PiCamZero automatically releases camera on object destruction
        print("Done.")


if __name__ == "__main__":
    main()

---

## نتیجه‌گیری

### دستاوردهای پروژه

1. **پیاده‌سازی کامل یک سیستم LPR**: از جمع‌آوری داده تا استقرار نهایی
2. **حل مشکل پلاک‌های خاص**: با افزایش داده هدفمند
3. **بهینه‌سازی برای embedded systems**: اجرا با 1.5 FPS بر روی RPi5
4. **معماری یکپارچه**: استفاده از یک مدل برای تمام وظایف

### محدودیت‌ها و کارهای آینده

1. **سرعت**: FPS بالاتر با استفاده از Coral TPU یا Jetson Nano
2. **دقت در شب**: افزودن داده‌های شبانه و استفاده از IR illumination
3. **پلاک‌های کثیف**: افزایش robustness با augmentation بیشتر
4. **ذخیره‌سازی**: اضافه کردن قابلیت ذخیره تصاویر و لاگ‌گیری

### سخن پایانی

این پروژه نشان می‌دهد که با استفاده از ابزارهای مدرن machine learning و کمی خلاقیت، می‌توان سیستم‌های practical و کاربردی را حتی با سخت‌افزار محدود پیاده‌سازی کرد. تجربه کار با Raspberry Pi و بهینه‌سازی مدل‌ها برای embedded systems، درس‌های ارزشمندی در مورد trade-off بین دقت، سرعت و منابع محاسباتی به ما آموخت.

توضیحات بصورت markdown و همچنین تمیزکردن کد برای خوانایی بیشتر بهمراه کامنت گذاری پس از انجام پروژه توسط هوش مصنوعی تدوین گردیده است.